In [2]:
'''
link for tutorial https://medium.com/analytics-vidhya/building-rgb-color-classifier-part-1-af58e3bcfef7
'''
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

In [3]:
dataset = pd.read_csv('../datas/RGB_color_labels.csv')
dataset

,red,green,blue,label
0,20,139,240,Blue
1,174,83,72,Brown
2,144,249,131,Green
3,168,25,156,Pink
4,30,182,136,Green
...,...,...,...,...
5047,26,26,26,Black
5048,27,27,27,Black
5049,28,28,28,Black
5050,29,29,29,Black


In [4]:
# One-Hot-Encoding
dataset = pd.get_dummies(dataset, columns=['label'])
dataset

,red,green,blue,label_Black,label_Blue,label_Brown,label_Green,label_Grey,label_Orange,label_Pink,label_Purple,label_Red,label_White,label_Yellow
0,20,139,240,0,1,0,0,0,0,0,0,0,0,0
1,174,83,72,0,0,1,0,0,0,0,0,0,0,0
2,144,249,131,0,0,0,1,0,0,0,0,0,0,0
3,168,25,156,0,0,0,0,0,0,1,0,0,0,0
4,30,182,136,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5047,26,26,26,1,0,0,0,0,0,0,0,0,0,0
5048,27,27,27,1,0,0,0,0,0,0,0,0,0,0
5049,28,28,28,1,0,0,0,0,0,0,0,0,0,0
5050,29,29,29,1,0,0,0,0,0,0,0,0,0,0


In [5]:
train_dataset = dataset.sample(frac=0.8, random_state=8) #train_dataset = 80% of total dataset  
#random_state = any int value means every time when you run your program you will get the same output for train and test dataset, random_state is None by default which means every time when you run your program you will get different output because of splitting between train and test varies within 
test_dataset = dataset.drop(train_dataset.index) #remove train_dataset from dataframe to get test_dataset

In [6]:
train_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    train_labels[x] = train_dataset.pop(x)
# train_labels = pd.DataFrame([train_dataset.pop(x) for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']]).T

In [7]:
train_labels

,label_Red,label_Green,label_Blue,label_Yellow,label_Orange,label_Pink,label_Purple,label_Brown,label_Grey,label_Black,label_White
4185,0,0,0,0,0,0,1,0,0,0,0
3378,0,0,0,1,0,0,0,0,0,0,0
2744,0,0,1,0,0,0,0,0,0,0,0
968,0,1,0,0,0,0,0,0,0,0,0
2286,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1883,0,0,0,0,0,0,1,0,0,0,0
3707,1,0,0,0,0,0,0,0,0,0,0
881,0,0,0,0,0,0,0,0,1,0,0
1352,0,0,0,0,0,0,1,0,0,0,0


In [8]:
test_labels = pd.DataFrame()
for x in ['label_Red', 'label_Green', 'label_Blue', 'label_Yellow', 'label_Orange', 'label_Pink', 'label_Purple', 'label_Brown', 'label_Grey', 'label_Black', 'label_White']:
    test_labels[x] = test_dataset.pop(x)

In [9]:
# model.build()
# model.summary()
labels = np.array(train_labels)
labels = np.argmax(labels, axis=1)
labels

array([6, 3, 2, ..., 8, 6, 3], dtype=int64)

In [10]:
# creating the layers
model = tf.keras.Sequential()

model.add(tf.keras.layers.Dense(len(train_dataset.keys()), kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=[len(train_dataset.keys())])) # Input layer

#9 layers #128 neurons each layer
for i in range(1,6):
    model.add(tf.keras.layers.Dense(128, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))

model.add(tf.keras.layers.Dense(11)) # Output layer

In [11]:
# settings for backprop
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [12]:
# saving model to a file
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 verbose=1,
                                                 save_best_only=True)

In [13]:
train_dataset

,red,green,blue
4185,189,148,192
3378,178,251,22
2744,54,9,125
968,119,147,60
2286,230,161,233
...,...,...,...
1883,223,157,205
3707,203,8,10
881,154,153,123
1352,151,141,236


In [14]:
model.fit(train_dataset, labels, batch_size=1000, epochs=100, shuffle=True, callbacks=[cp_callback])

Epoch 1/100
5/5 [==============================] - 2s 12ms/step - loss: 3.8646 - accuracy: 0.2526
Epoch 2/100
5/5 [==============================] - 0s 4ms/step - loss: 2.1361 - accuracy: 0.5317
Epoch 3/100
5/5 [==============================] - 0s 4ms/step - loss: 1.6856 - accuracy: 0.6257
Epoch 4/100
5/5 [==============================] - 0s 4ms/step - loss: 1.4217 - accuracy: 0.6655
Epoch 5/100
5/5 [==============================] - 0s 4ms/step - loss: 1.2260 - accuracy: 0.7261
Epoch 6/100
5/5 [==============================] - 0s 4ms/step - loss: 1.1335 - accuracy: 0.7326
Epoch 7/100
5/5 [==============================] - 0s 4ms/step - loss: 1.0775 - accuracy: 0.7464
Epoch 8/100
5/5 [==============================] - 0s 4ms/step - loss: 1.0256 - accuracy: 0.7387
Epoch 9/100
5/5 [==============================] - 0s 4ms/step - loss: 1.0042 - accuracy: 0.7526
Epoch 10/100
5/5 [==============================] - 0s 5ms/step - loss: 0.9955 - accuracy: 0.7521
Epoch 11/100
5/5 [==========

In [15]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])
num = 9
print(probability_model.predict(train_dataset)[num])
train = np.array(train_labels)
print(train[num])

127/127 [==============================] - 0s 945us/step
[6.2703393e-06 3.2366099e-04 6.2845075e-01 5.3157551e-06 1.4294245e-06
 1.1001457e-03 2.9959571e-01 4.5154770e-06 7.0175439e-02 2.7070395e-04
 6.6115179e-05]
[0 0 1 0 0 0 0 0 0 0 0]


In [16]:
test_labels_flattened = np.array(test_labels)
test_labels_flattened = np.argmax(test_labels_flattened, axis=1)
model.evaluate(test_dataset, test_labels_flattened)

32/32 [==============================] - 0s 1ms/step - loss: 0.6925 - accuracy: 0.8376


[0.6924630403518677, 0.8376237750053406]

In [17]:
latest = tf.train.latest_checkpoint(checkpoint_dir)

In [18]:
# creating the layers
load_model = tf.keras.Sequential()

load_model.add(tf.keras.layers.Dense(len(train_dataset.keys()), kernel_regularizer=keras.regularizers.l2(0.001), activation='relu', input_shape=[len(train_dataset.keys())])) # Input layer

#9 layers #128 neurons each layer
for i in range(1,10):
    load_model.add(tf.keras.layers.Dense(128, kernel_regularizer=keras.regularizers.l2(0.001), activation='relu'))

load_model.add(tf.keras.layers.Dense(11)) # Output layer

# Load the previously saved weights
load_model.load_weights(latest)

# Re-evaluate the model
loss, acc = load_model.evaluate(test_dataset, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

AttributeError: 'NoneType' object has no attribute 'endswith'

In [ ]:
probability_model = tf.keras.Sequential()
probability_model.add(model)
probability_model.add(tf.keras.layers.Softmax())

predictions = probability_model.predict(test_dataset)

32/32 [==============================] - 0s 4ms/step


In [ ]:
displayed_ans = 90
print(predictions[displayed_ans])
print(test_labels_flattened[displayed_ans])

[1.1755257e-08 8.2709048e-06 6.0667619e-03 3.8124912e-10 2.5643687e-10
 5.6250388e-04 6.1787713e-01 2.4186734e-03 1.7599579e-02 3.5546693e-01
 1.8917117e-08]
6
